In [1]:
import pandas as pd
import numpy as np

In [3]:
pd.read_csv('data/rand.csv')

,Unnamed: 0,date,amount,bal
0,0,2017-01-29,60.00,696.40
1,1,2017-01-29,200.00,896.40
2,2,2017-01-29,-39.90,856.50
3,3,2017-01-29,-95.00,761.50
4,4,2017-01-30,410.00,1171.50
5,5,2017-01-30,-410.00,761.50
6,6,2017-01-30,-40.15,721.35
7,7,2017-01-30,-25.85,695.50
8,8,2017-01-31,-43.50,652.00
9,9,2017-01-31,-16.59,635.41
